In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max.columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from bayes_opt import BayesianOptimization

import matplotlib.pyplot as plt
plt.rc("font", family="NanumGothic") # 라이브러리 불러오기와 함께 한번만 실행

In [2]:
path = './data/'
liq = pd.read_csv(path + './액체_모델용.csv')
n_liq = pd.read_csv(path + './비액체_모델용.csv')
df_all = pd.read_csv(path + './울산_전처리_ver7.csv')

In [3]:
liq.head()

df = liq.copy()
df = df.sort_values(by = ['ETA_Year', 'ETA_Month', 'ETA_Day', 'ETA_Hour'])
df.head(3)

,Datetime,ETA_Year,ETA_Month,ETA_Day,ETA_Hour,호출부호,입항일시,출항일시,접안_대기시간_분,출항_대기시간_분,Service_Time_Start,Service_Time_End,계선장소_코드,계선장소_숫자,계선장소명,선박용도,Service_Time_분,대기율,총톤수,재화중량톤수,"선박제원(총길이, m)","선박제원(너비, m)",만재흘수(m),깊이(m),길이1(m),ETA_Minute,풍속(m/s),풍향(deg),GUST풍속(m/s),현지기압(hPa),습도(%),기온(°C),수온(°C),최대파고(m),유의파고(m),평균파고(m),파주기(sec),파향(deg),액체화물
0,2020-01-01 01:00:00,2020,1,1,1,88238,2020-01-01 01:05:00,2020-01-01 08:15,0,0.0,2020-01-01 01:05:00,2020-01-01 08:15:00,MBU,1,SK2부두 01,석유제품 운반선,430.0,0.0,700.0,0.0,70.9,12.0,3.9,5.35,66.01,5,9.3,330.0,12.2,1028.8,43.0,4.3,17.5,4.2,2.8,2.0,10.7,49.0,1
1,2020-01-01 03:00:00,2020,1,1,3,DSRB9,2020-01-01 03:05:00,2020-01-01 14:55,0,0.0,2020-01-01 03:05:00,2020-01-01 14:55:00,MBL,1,가스부두,LPG 운반선,710.0,0.0,2689.0,2999.0,96.0,15.0,7.2,7.20,90.10,5,12.6,315.0,15.3,1028.5,44.0,4.3,17.4,3.9,2.2,1.5,9.1,48.0,1
2,2020-01-01 04:00:00,2020,1,1,4,D7SC,2020-01-01 04:30:00,2020-01-01 20:40,0,0.0,2020-01-01 04:30:00,2020-01-01 20:40:00,MBU,12,SK1부두 12,석유제품 운반선,970.0,0.0,1130.0,2286.0,83.3,14.0,5.0,5.95,74.00,30,11.4,317.0,14.4,1028.3,53.0,3.4,17.3,3.5,2.1,1.5,9.1,56.0,1


In [4]:
df = df.drop(columns = ['출항_대기시간_분', 'Service_Time_Start', 'Service_Time_End', '계선장소_코드', '계선장소_숫자', '액체화물'])

In [5]:
df.columns

Index(['Datetime', 'ETA_Year', 'ETA_Month', 'ETA_Day', 'ETA_Hour', '호출부호',
       '입항일시', '출항일시', '접안_대기시간_분', '계선장소명', '선박용도', 'Service_Time_분', '대기율',
       '총톤수', '재화중량톤수', '선박제원(총길이, m)', '선박제원(너비, m)', '만재흘수(m)', '깊이(m)',
       '길이1(m)', 'ETA_Minute', '풍속(m/s)', '풍향(deg)', 'GUST풍속(m/s)',
       '현지기압(hPa)', '습도(%)', '기온(°C)', '수온(°C)', '최대파고(m)', '유의파고(m)',
       '평균파고(m)', '파주기(sec)', '파향(deg)'],
      dtype='object')

## [변수1] 선박 일 별 누적입항 건수

In [9]:
df['이전_입항선박수'] = df.groupby(['ETA_Year', 'ETA_Month', 'ETA_Day', '계선장소명'])['호출부호'].cumcount()

# 확인용
df[df['계선장소명']=='가스부두'][['Datetime', '계선장소명', '호출부호', '이전_입항선박수']].head(60)

,Datetime,계선장소명,호출부호,이전_입항선박수
1,2020-01-01 03:00:00,가스부두,DSRB9,0
5,2020-01-01 05:00:00,가스부두,3FSX4,1
29,2020-01-01 17:00:00,가스부두,134806,2
30,2020-01-01 17:00:00,가스부두,D5NP9,3
51,2020-01-02 09:00:00,가스부두,3ESC2,0
61,2020-01-02 14:00:00,가스부두,BYIN,1
64,2020-01-02 14:00:00,가스부두,DSRN9,2
94,2020-01-03 04:00:00,가스부두,180016,0
109,2020-01-03 11:00:00,가스부두,DSPH2,1
132,2020-01-03 20:00:00,가스부두,3FIG4,2


## [변수2] 3년 평균 선석 점유율

In [11]:
# 선석점유율 계산시 워킹 데이 계산 다시함(3년 각각)
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Month_Day'] = df['Datetime'].dt.strftime('%m-%d')

# 연도별/계선장소별 서비스 시간 확인 필요 : 3년 각각
temp2 = df.groupby(['ETA_Year', '계선장소명']).agg({'Service_Time_분':'sum', 'Month_Day':'nunique'}).reset_index()

# 연도별/계선장소별, 선석점유율계산
temp2['연도별_선석점유율'] = temp2['Service_Time_분'] / (temp2['Month_Day'] * 24 * 60)
temp2.head()

# 3년 평균 선석점유율
temp3 = temp2.groupby('계선장소명')['연도별_선석점유율'].mean().to_frame().reset_index()
temp3

,계선장소명,연도별_선석점유율
0,(구)벙커링전용정박지 E1,2.105879
1,(구)벙커링전용정박지 E2,2.438987
2,(구)벙커링전용정박지 E3,3.094719
3,1부두 01,1.217967
4,2부두 01,1.292531
...,...,...
120,현중해양의장안벽,2.725798
121,화암부두 01,0.722222
122,화암부두 05,0.506944
123,화암부두 06,0.822396


In [12]:
# Merge
print(df.shape)
df = df.merge(temp3, on = '계선장소명', how = 'left')
print(df.shape)
df.head(3)

(51001, 35)
(51001, 36)


,Datetime,ETA_Year,ETA_Month,ETA_Day,ETA_Hour,호출부호,입항일시,출항일시,접안_대기시간_분,계선장소명,선박용도,Service_Time_분,대기율,총톤수,재화중량톤수,"선박제원(총길이, m)","선박제원(너비, m)",만재흘수(m),깊이(m),길이1(m),ETA_Minute,풍속(m/s),풍향(deg),GUST풍속(m/s),현지기압(hPa),습도(%),기온(°C),수온(°C),최대파고(m),유의파고(m),평균파고(m),파주기(sec),파향(deg),이전_입항선박수,Month_Day,연도별_선석점유율
0,2020-01-01 01:00:00,2020,1,1,1,88238,2020-01-01 01:05:00,2020-01-01 08:15,0,SK2부두 01,석유제품 운반선,430.0,0.0,700.0,0.0,70.9,12.0,3.9,5.35,66.01,5,9.3,330.0,12.2,1028.8,43.0,4.3,17.5,4.2,2.8,2.0,10.7,49.0,0,01-01,4.143219
1,2020-01-01 03:00:00,2020,1,1,3,DSRB9,2020-01-01 03:05:00,2020-01-01 14:55,0,가스부두,LPG 운반선,710.0,0.0,2689.0,2999.0,96.0,15.0,7.2,7.20,90.10,5,12.6,315.0,15.3,1028.5,44.0,4.3,17.4,3.9,2.2,1.5,9.1,48.0,0,01-01,1.742879
2,2020-01-01 04:00:00,2020,1,1,4,D7SC,2020-01-01 04:30:00,2020-01-01 20:40,0,SK1부두 12,석유제품 운반선,970.0,0.0,1130.0,2286.0,83.3,14.0,5.0,5.95,74.00,30,11.4,317.0,14.4,1028.3,53.0,3.4,17.3,3.5,2.1,1.5,9.1,56.0,0,01-01,2.054285


## [변수3] 선석의 3년 평균 톤 처리량

- 연간 톤 처리량 
    - 선석별 : 재화중량 톤 (3년 평균)

In [13]:
temp = df.copy()

# 연도별, 계선장소별 재화중량톤수 평균 계산
temp_mean = temp.groupby(['ETA_Year', '계선장소명'])['재화중량톤수'].mean().reset_index()

# 각 계선장소별로 3년 평균 계산
result = temp_mean.groupby('계선장소명')['재화중량톤수'].mean().to_frame().reset_index()

result.columns = ['계선장소명', '시설연평균_재화중량톤수']

In [14]:
# 연간 톤 처리량 merge
print(df.shape)
df = df.merge(result, on = '계선장소명', how = 'left')
print(df.shape)

(51001, 36)
(51001, 37)


## [변수4] 선석의 3년 평균 입항 척수
    - 연도별/선석별 입항일시 건수 count()

In [15]:
ship_cnt = df.groupby(['ETA_Year', '계선장소명'])['입항일시'].count().reset_index()
ship_cnt = ship_cnt.rename(columns = {'입항일시':'연간_총입항건수'})
# 각 계선장소별로 3년 평균 계산
ship_cnt_result = ship_cnt.groupby('계선장소명')['연간_총입항건수'].mean().to_frame().reset_index()
ship_cnt_result = ship_cnt_result.rename(columns = {'연간_총입항건수':'연평균_총입항건수'})
ship_cnt_result

,계선장소명,연평균_총입항건수
0,(구)벙커링전용정박지 E1,244.500000
1,(구)벙커링전용정박지 E2,127.000000
2,(구)벙커링전용정박지 E3,68.000000
3,1부두 01,21.333333
4,2부두 01,24.333333
...,...,...
120,현중해양의장안벽,37.000000
121,화암부두 01,2.000000
122,화암부두 05,1.000000
123,화암부두 06,2.500000


In [16]:
print(df.shape)
df = df.merge(ship_cnt_result, on = '계선장소명', how = 'left')
print(df.shape)

(51001, 37)
(51001, 38)


## [변수5] 선박의 3년 평균 서비스 시간
- 입항선박접안시간(합)/처리 접안(서비스)척수
    - 선박 1건 기준
    - 서비스시간 합 
    - 서비스기간 발생 건 합

In [18]:
temp = df.groupby(['ETA_Year', '호출부호']).agg({'Service_Time_분' : ['sum', 'count']}).reset_index()
temp['선박_평균서비스시간'] = temp['Service_Time_분']['sum'] / temp['Service_Time_분']['count']
temp = temp.groupby('호출부호')['선박_평균서비스시간'].mean().to_frame().reset_index().rename(columns = {'선박_평균서비스시간':'선박_연평균_서비스시간'})
temp.head()

,호출부호,선박_연평균_서비스시간
0,100314,57953.805556
1,100611,979.171441
2,100830,3280.509819
3,10192,320.677134
4,101991,1149.851852


In [19]:
print(df.shape)
df = df.merge(temp, on = '호출부호', how = 'left')
print(df.shape)

(51001, 38)
(51001, 39)


## [변수6] 선박 1건당 3년 평균 대기시간
    - 선박 1건
    - 대기시간 합
    - 대기 건수

In [20]:
temp = df.groupby(['ETA_Year', '호출부호']).agg({'접안_대기시간_분' : ['sum', 'count']}).reset_index()
temp['선박_평균대기시간'] = temp['접안_대기시간_분']['sum'] / temp['접안_대기시간_분']['count']
temp = temp.groupby('호출부호')['선박_평균대기시간'].mean().to_frame().reset_index().rename(columns = {'선박_평균대기시간':'선박_연평균_대기시간'})
temp

,호출부호,선박_연평균_대기시간
0,100314,0.000000
1,100611,321.410429
2,100830,1011.694312
3,10192,1.481481
4,101991,842.629630
...,...,...
3076,ZGHP9,495.000000
3077,ZGHZ4,9918.000000
3078,ZGIH9,0.000000
3079,ZIRQ6,1789.000000


In [21]:
print(df.shape)
df = df.merge(temp, on = '호출부호', how = 'left')
print(df.shape)

(51001, 39)
(51001, 40)


In [22]:
df.head(2)

,Datetime,ETA_Year,ETA_Month,ETA_Day,ETA_Hour,호출부호,입항일시,출항일시,접안_대기시간_분,계선장소명,선박용도,Service_Time_분,대기율,총톤수,재화중량톤수,"선박제원(총길이, m)","선박제원(너비, m)",만재흘수(m),깊이(m),길이1(m),ETA_Minute,풍속(m/s),풍향(deg),GUST풍속(m/s),현지기압(hPa),습도(%),기온(°C),수온(°C),최대파고(m),유의파고(m),평균파고(m),파주기(sec),파향(deg),이전_입항선박수,Month_Day,연도별_선석점유율,시설연평균_재화중량톤수,연평균_총입항건수,선박_연평균_서비스시간,선박_연평균_대기시간
0,2020-01-01 01:00:00,2020,1,1,1,88238,2020-01-01 01:05:00,2020-01-01 08:15,0,SK2부두 01,석유제품 운반선,430.0,0.0,700.0,0.0,70.9,12.0,3.9,5.35,66.01,5,9.3,330.0,12.2,1028.8,43.0,4.3,17.5,4.2,2.8,2.0,10.7,49.0,0,01-01,4.143219,2073.888986,1729.000000,1452.987426,572.048817
1,2020-01-01 03:00:00,2020,1,1,3,DSRB9,2020-01-01 03:05:00,2020-01-01 14:55,0,가스부두,LPG 운반선,710.0,0.0,2689.0,2999.0,96.0,15.0,7.2,7.20,90.10,5,12.6,315.0,15.3,1028.5,44.0,4.3,17.4,3.9,2.2,1.5,9.1,48.0,0,01-01,1.742879,3935.463021,418.333333,970.370370,215.740741


In [23]:
ship_model = df.drop(['Month_Day', 'ETA_Minute', 'Datetime', '입항일시', '출항일시'], axis = 1)

In [24]:
ship_model.head()

,ETA_Year,ETA_Month,ETA_Day,ETA_Hour,호출부호,접안_대기시간_분,계선장소명,선박용도,Service_Time_분,대기율,총톤수,재화중량톤수,"선박제원(총길이, m)","선박제원(너비, m)",만재흘수(m),깊이(m),길이1(m),풍속(m/s),풍향(deg),GUST풍속(m/s),현지기압(hPa),습도(%),기온(°C),수온(°C),최대파고(m),유의파고(m),평균파고(m),파주기(sec),파향(deg),이전_입항선박수,연도별_선석점유율,시설연평균_재화중량톤수,연평균_총입항건수,선박_연평균_서비스시간,선박_연평균_대기시간
0,2020,1,1,1,88238,0,SK2부두 01,석유제품 운반선,430.0,0.000000,700.0,0.0,70.9,12.0,3.9,5.35,66.01,9.3,330.0,12.2,1028.8,43.0,4.3,17.5,4.2,2.8,2.0,10.7,49.0,0,4.143219,2073.888986,1729.000000,1452.987426,572.048817
1,2020,1,1,3,DSRB9,0,가스부두,LPG 운반선,710.0,0.000000,2689.0,2999.0,96.0,15.0,7.2,7.20,90.10,12.6,315.0,15.3,1028.5,44.0,4.3,17.4,3.9,2.2,1.5,9.1,48.0,0,1.742879,3935.463021,418.333333,970.370370,215.740741
2,2020,1,1,4,D7SC,0,SK1부두 12,석유제품 운반선,970.0,0.000000,1130.0,2286.0,83.3,14.0,5.0,5.95,74.00,11.4,317.0,14.4,1028.3,53.0,3.4,17.3,3.5,2.1,1.5,9.1,56.0,0,2.054285,3055.315947,184.666667,1936.539220,130.389610
3,2020,1,1,5,130095,0,정박지-M1,석유제품 운반선,1760.0,0.000000,1130.0,2286.0,79.0,14.0,5.0,5.95,74.00,9.9,305.0,13.5,1028.9,60.0,3.0,16.8,4.0,2.1,1.5,8.0,38.0,0,1.555326,2462.584826,356.000000,1124.078947,959.210526
4,2020,1,1,5,160014,517,현대오일터미널신항부두,석유제품 운반선,743.0,69.582773,1615.0,2854.0,82.5,13.0,3.1,6.50,78.29,9.9,305.0,13.5,1028.9,60.0,3.0,16.8,4.0,2.1,1.5,8.0,38.0,0,1.669914,2942.975883,901.666667,703.759259,569.314815


- 모델링을 위한 카테고리 컬럼 인코딩

In [25]:
#ship_model encoding
obj_cols = ['호출부호', '계선장소명', '선박용도']
for col in obj_cols:
    ship_model[f'{col}_encoded'] = pd.factorize(ship_model[col])[0]

In [26]:
ship_model.to_csv('for_ship_model.csv', index = False, encoding = 'utf-8-sig')

In [27]:
ship_model.head()

,ETA_Year,ETA_Month,ETA_Day,ETA_Hour,호출부호,접안_대기시간_분,계선장소명,선박용도,Service_Time_분,대기율,총톤수,재화중량톤수,"선박제원(총길이, m)","선박제원(너비, m)",만재흘수(m),깊이(m),길이1(m),풍속(m/s),풍향(deg),GUST풍속(m/s),현지기압(hPa),습도(%),기온(°C),수온(°C),최대파고(m),유의파고(m),평균파고(m),파주기(sec),파향(deg),이전_입항선박수,연도별_선석점유율,시설연평균_재화중량톤수,연평균_총입항건수,선박_연평균_서비스시간,선박_연평균_대기시간,호출부호_encoded,계선장소명_encoded,선박용도_encoded
0,2020,1,1,1,88238,0,SK2부두 01,석유제품 운반선,430.0,0.000000,700.0,0.0,70.9,12.0,3.9,5.35,66.01,9.3,330.0,12.2,1028.8,43.0,4.3,17.5,4.2,2.8,2.0,10.7,49.0,0,4.143219,2073.888986,1729.000000,1452.987426,572.048817,0,0,0
1,2020,1,1,3,DSRB9,0,가스부두,LPG 운반선,710.0,0.000000,2689.0,2999.0,96.0,15.0,7.2,7.20,90.10,12.6,315.0,15.3,1028.5,44.0,4.3,17.4,3.9,2.2,1.5,9.1,48.0,0,1.742879,3935.463021,418.333333,970.370370,215.740741,1,1,1
2,2020,1,1,4,D7SC,0,SK1부두 12,석유제품 운반선,970.0,0.000000,1130.0,2286.0,83.3,14.0,5.0,5.95,74.00,11.4,317.0,14.4,1028.3,53.0,3.4,17.3,3.5,2.1,1.5,9.1,56.0,0,2.054285,3055.315947,184.666667,1936.539220,130.389610,2,2,0
3,2020,1,1,5,130095,0,정박지-M1,석유제품 운반선,1760.0,0.000000,1130.0,2286.0,79.0,14.0,5.0,5.95,74.00,9.9,305.0,13.5,1028.9,60.0,3.0,16.8,4.0,2.1,1.5,8.0,38.0,0,1.555326,2462.584826,356.000000,1124.078947,959.210526,3,3,0
4,2020,1,1,5,160014,517,현대오일터미널신항부두,석유제품 운반선,743.0,69.582773,1615.0,2854.0,82.5,13.0,3.1,6.50,78.29,9.9,305.0,13.5,1028.9,60.0,3.0,16.8,4.0,2.1,1.5,8.0,38.0,0,1.669914,2942.975883,901.666667,703.759259,569.314815,4,4,0
